# Nextcloud on Arch Linux
I recently had to re-install nextcloud on my arch linux server. I decided it was time to finally document this process. It's not so difficult, but there are a lot of moving parts that need to come together.

## Installing dependencies

These are all the packages you'll need to install. Go ahead and install them first with your AUR helper (like yay), we'll go over the configuration later.

```bash
apache
imagemagick
smbclient
mariadb
ffmpeg
libreoffice
php
php-gd
php-imagick
php-memcached
php-apcu
php-imap
php-smbclient
php-intl
php-fpm
nextcloud
certbot
certbot-apache
```

## MariaDB

### Base mariadb installation

Mariadb is the FOSS version of MySQL. Install and enable as follows:
```bash
    sudo mariadb-install-db --user=mysql --basedir=/usr --datadir=/var/lib/mysql
    sudo systemctl enable mariadb
    sudo systemctl start mariadb
```

This enables the mariadb service, however, it's not yet secure. Run the following to make your mariadb installation more secure:
```bash
    sudo mysql_secure_installation
```

There's no password set for the root user, so just press enter when asked for a password. Then, choose the following options for the rest of the installer.
```bash
Switch to unix_socket authentication [Y/n] y
Change the root password? [Y/n] n
Remove anonymous users? [Y/n] y
Disallow root login remotely? [Y/n] y
Remove test database and access to it? [Y/n] y
Reload privilege tables now? [Y/n] y
```

config file is located at `/etc/my.cnf`

Then edit `/etc/my.cnf.d/server.cnf` and make sure the following `mysqld` section exists:

```ini
[mysqld]
skip-networking = 1
log-bin = mysql-bin
binlog_format = mixed
```

We also need to add a compatibility flag for newer mariadb versions:

```ini
[mariadb-10.6]
innodb_read_only_compressed=OFF
```

### Nextcloud database

Create a `nextcloud` user by first starting mysql with the mysql root account:

```bash
    sudo mysql -u root
```

Then, within the SQL prompt, create a new user called "nextcloud":
```sql
    create user nextcloud@localhost identified by '<password>';
```

FYI, you can see all users with the following SQL command:
```sql
    select user from mysql.user;
```

Next, create an empty database called "nextcloud":
```sql
    create database nextcloud;
    grant all privileges on nextcloud.* to nextcloud@localhost identified by '<password>';
    flush privileges;
```

FYI, you can see all databases with the following SQL command:

```sql
    show databases;
```

Exit the sql prompt:

```sql
    exit;
```

For a new nextcloud installation, that's it. Your database is now correctly configured. However, when migrating your nextcloud installation from one server to another, do the following two steps as well:

First, **on the old server**, export your "nextcloud" database (assuming user "nextcloud" and database "nextcloud"):
```bash
    sudo mysqldump -u nextcloud -p --opt nextcloud > exported_database.sql
```
Copy `exported_database.sql` to the new server.

Then, **on the new server**, import `exported_database.sql` into the newly created empty "nextcloud" database as follows:
```bash
    sudo mysql -u root nextcloud < exported_database.sql
```

Note that this might take a considerable amount of time for large databases. However, this should be enough to set up MariaDB database correctly.

## PHP

### Extensions

All dependencies should already have been installed in the "Install Dependencies" section. Now enable all php packages you need by editing the 'Dynamic Extensions' section in `/etc/php/php.ini`. Make sure the following packages are uncommented/added:

```ini
;;;;;;;;;;;;;;;;;;;;;;
; Dynamic Extensions ;
;;;;;;;;;;;;;;;;;;;;;;

extension=apcu
extension=bcmath
extension=bz2
;extension=calendar
extension=curl
;extension=dba
;extension=enchant
extension=exif
;extension=ffi
extension=ftp
extension=gd
;extension=gettext
extension=gmp
extension=iconv
extension=imagick
extension=imap
extension=intl
extension=ldap
extension=memcached
extension=mysqli
;extension=odbc
zend_extension=opcache
;extension=pdo_dblib
extension=pdo_mysql
;extension=pdo_odbc
;extension=pdo_pgsql
;extension=pdo_sqlite
;extension=pgsql
;extension=pspell
;extension=shmop
;extension=snmp
;extension=soap
;extension=sockets
;extension=sodium
;extension=sqlite3
;extension=sysvmsg
;extension=sysvsem
;extension=sysvshm
;extension=tidy
;extension=xmlrpc
;extension=xsl
extension=zip
```

Also, don't forget to set your timezone in `/etc/php/php.ini`. For example:

```
date.timezone = Europe/Berlin
```

And set the `session.save_path` variable:

```ini
session.save_path = "/tmp"
```

We also want to increase the maximum opload size:

```ini
upload_max_filesize = 25M
```

Finally, you'll need to set the `fix_pathinfo` flag:

```ini
cgi.fix_pathinfo=1
```

### OPCache

Next, we need to improve the caching. Edit the `[opcache]` section in `/etc/php/php.ini`:

```ini
[opcache]
opcache.enable = 1
opcache.memory_consumption = 512
opcache.interned_strings_buffer = 16
opcache.max_accelerated_files = 10000
opcache.revalidate_freq = 1
opcache.save_comments = 1
```

Then increase `memory_limit` (somewhere in the same file) to `512M`.

### APCU

We also want to enable apcu caching. Add the following to the end of `/etc/php/php.ini`:

```ini
[apcu]
apc.enabled=1
apc.shm_size=32M
apc.ttl=7200
apc.enable_cli=1
```

### PHP-FPM

create `/etc/php/php-fpm.d/nextcloud.conf` and add the following content:

```ini
[nextcloud]
user = nextcloud
group = nextcloud
listen = /run/nextcloud/nextcloud.sock
env[PATH] = /usr/local/bin:/usr/bin:/bin:/usr/local/sbin
env[TMP] = /tmp

listen.owner = http
listen.group = http

pm = dynamic
pm.max_children = 120
pm.start_servers = 12
pm.min_spare_servers = 6
pm.max_spare_servers = 18
```

Then edit the php-fpm service
```bash
    sudo systemctl edit php-fpm
```
 
 And add the path to your nextcloud data directory as follows:
 
```ini
[Service]
# your nextcloud data directory
ReadWritePaths = /var/lib/nextcloud/data

# your nextcloud apps directory
ReadWritePaths = /var/lib/nextcloud/apps

# your config directory
ReadWritePaths=/etc/webapps/nextcloud/config
```

Finally, enable and start the `php-fpm` service:
```bash
    sudo systemctl enable php-fpm
    sudo systemctl restart php-fpm
```

## Nextcloud server installation

First, edit your nextcloud configuration file at `/etc/webapps/nextcloud/config/config.php`. Here you can change the location of your data folder and the location where apps will be installed. The default locations are:

- datadirectory: `/var/lib/nextcloud/data`
- apps: `/var/lib/nextcloud/apps`

We won't be changing these locations, however make sure they match the `ReadWritePaths` in your php-fpm service. We should also add the following line to enable better APCu caching:

```php
'memcache.local' => '\OC\Memcache\APCu',
```

This will prevent nextcloud from overriding your settings. The only way to update the settings is by editing the file manually from now on.

Finally, we want to prevent installing updates for nextcloud via pacman (we can always manually update it by explicitly reinstalling). Edit `/etc/pacman.conf` and add:

```ini
IgnorePkg = nextcloud
```

### Note

If you're reinstalling nextcloud, be sure to use your old config file!

### Cron Jobs

We also need to install a cron job for nextcloud. We can do this using systemd timers. 

First, create `/etc/systemd/system/nextcloudcron.service`:

```ini
[Unit]
Description=nextcloud cron.php job

[Service]
User=nextcloud
ExecStart=/usr/bin/php -f /usr/share/webapps/nextcloud/cron.php
```

Then create `/etc/systemd/system/nextcloudcron.timer`:

```ini
[Unit]
Description=Run Nextcloud cron.php every 5 minutes

[Timer]
OnBootSec=5min
OnUnitActiveSec=5min
Unit=nextcloudcron.service

[Install]
WantedBy=timers.target
```

then enable and start the timer

```
sudo systemctl enable nextcloudcron.timer
sudo systemctl start nextcloudcron.timer
```

## Domain name

Make sure you have a domain name associated to your ip address. If you're currently working in a local setup and do not have a domain associated with your account you can edit `/etc/hosts` to add a *fake* link between a domain and the local host. Let's assume we're using the `nextcloud.example.com` domain, add this to your `/etc/hosts`:

```
127.0.0.1 nextcloud.example.com
```

**NOTE: do this only if you do not have an actual domain pointing to your public ip!**

Now, add this host to your nextcloud config at `/etc/webapps/nextcloud/config/config.php` by adding the following `'trusted_domains'`:

```php
'trusted_domains' => 
array (
  0 => 'nextcloud.example.com',
)
```

You can add as many trusted domains as you want. Nextcloud will refuse to serve to any domains not in this list.

## Apache config

Finally, we need to configure Apache (you can also use nginx, but in general I feel like Apache just works better and is easier to configure). Copy the default apache configuration file into `/etc/httpd/conf/extra`:

```bash
sudo cp /usr/share/doc/nextcloud/apache.example.conf /etc/httpd/conf/extra/nextcloud.conf
```

Then change the servername in `/etc/httpd/conf/extra/nextcloud.conf`:

```xml
<VirtualHost *:80>
    ...
    ServerName nextcloud.example.com
    ...
</VirtualHost>
```

And add the following configuration to the end of `/etc/httpd/conf/extra/nextcloud.conf`:
```xml
DirectoryIndex index.php index.html
<FilesMatch \.php$>
    SetHandler "proxy:unix:/run/nextcloud/nextcloud.sock|fcgi://localhost/"
</FilesMatch>
```

Then edit the main apache config at `/etc/httpd/conf/httpd.conf`:

Uncomment the following modules:

```xml
LoadModule socache_shmcb_module modules/mod_socache_shmcb.so
LoadModule proxy_module modules/mod_proxy.so
LoadModule proxy_fcgi_module modules/mod_proxy_fcgi.so
LoadModule ssl_module modules/mod_ssl.so
LoadModule http2_module modules/mod_http2.so
LoadModule rewrite_module modules/mod_rewrite.so
```

And include the nextcloud configuration at the end of the same file:

```xml
Include conf/extra/nextcloud.conf
```

## HTTPS config

**NOTE: this only works for actual domain names pointing to your IP address, if you hacked the hostname into /etc/hosts, this will not work**

Configuring https is easiest with certbot:
```bash
sudo certbot --apache -d nextcloud.example.com
```

Answer the questions of the wizard and let certbot modify your apache configuration automatically. If everything went well, the nextcloud apache config at `/etc/httpd/conf/extra/nextcloud.conf` should now have a new section for port 443 (the https port). Add the following recommended [HSTS settings](https://docs.nextcloud.com/server/22/admin_manual/installation/harden_server.html#enable-http-strict-transport-security) to it:

```xml
<VirtualHost *:443>
    ...
    <IfModule mod_headers.c>
        Header always set Strict-Transport-Security "max-age=15552000; includeSubDomains"
    </IfModule>
</VirtualHost>
```


Nextcloud should now be correctly installed!

## First nextcloud connection
Now go to nextcloud.example.com and create an admin user and password and connect to the mariadb nextcloud database. Once this is finished, nextcloud is completely set up!